In [1]:
import xarray as xr
import numpy as np
from dask import array as darr
from dask.distributed import Client

In [2]:
client = Client('tcp://127.0.0.1:43049')
client

<Client: 'tcp://127.0.0.1:43049' processes=4 threads=16, memory=64.00 GiB>

In [3]:
# parameters
num_nodes = 22_000
chunks_nodes = 2_000
num_neighbors = 10
len_time = 4_000
chunks_time = 40
len_z = 40
chunks_z = 10

In [4]:
n = xr.DataArray(
    np.arange(num_nodes, ),
    dims=("n", ),
    name="n",
)
n

<xarray.DataArray 'n' (n: 22000)>
array([    0,     1,     2, ..., 21997, 21998, 21999])
Dimensions without coordinates: n

In [5]:
neighbors = xr.DataArray(
    np.arange(num_neighbors),
    dims=("j", ),
    name="neighbor",
)

In [6]:
z = xr.DataArray(
    np.arange(len_z),
    dims=("z", ),
    name="z",
)
z

<xarray.DataArray 'z' (z: 40)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])
Dimensions without coordinates: z

In [23]:
groups = (
    n.isel(n=slice(None, None, num_neighbors))
    + neighbors
).rename("groups").rename({"n": "m"})

groups

<xarray.DataArray 'groups' (m: 2200, j: 10)>
array([[    0,     1,     2, ...,     7,     8,     9],
       [   10,    11,    12, ...,    17,    18,    19],
       [   20,    21,    22, ...,    27,    28,    29],
       ...,
       [21970, 21971, 21972, ..., 21977, 21978, 21979],
       [21980, 21981, 21982, ..., 21987, 21988, 21989],
       [21990, 21991, 21992, ..., 21997, 21998, 21999]])
Dimensions without coordinates: m, j

In [24]:
groups.to_dataset().to_netcdf("03_fesom_coarse_graining_dask_nc_groups.nc")

In [25]:
T = xr.DataArray(
    darr.random.normal(
        size=(num_nodes, len_time, len_z),
        chunks=(chunks_nodes, chunks_time, chunks_z),
    ),
    dims=("n", "t", "z"),
    name="temp",
    coords={"n": n, "z": z},
)
T

<xarray.DataArray 'temp' (n: 22000, t: 4000, z: 40)>
dask.array<normal, shape=(22000, 4000, 40), dtype=float64, chunksize=(2000, 40, 10), chunktype=numpy.ndarray>
Coordinates:
  * n        (n) int64 0 1 2 3 4 5 6 ... 21994 21995 21996 21997 21998 21999
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: t

In [26]:
T.to_dataset().to_zarr("03_fesom_coarse_graining_dask_nc_T.zarr", mode="w")

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 11.39 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [27]:
W = xr.DataArray(
    darr.ones(
        shape=(num_nodes, num_neighbors),
        chunks=(chunks_nodes, num_neighbors),
    ),
    dims=("n", "j"),
    name="weights",
)
W = W.assign_coords(n=n)
W

<xarray.DataArray 'weights' (n: 22000, j: 10)>
dask.array<ones_like, shape=(22000, 10), dtype=float64, chunksize=(2000, 10), chunktype=numpy.ndarray>
Coordinates:
  * n        (n) int64 0 1 2 3 4 5 6 ... 21994 21995 21996 21997 21998 21999
Dimensions without coordinates: j

In [28]:
W.to_dataset().to_zarr("03_fesom_coarse_graining_dask_nc_W.zarr/", mode="w")

In [29]:
T = xr.open_zarr("03_fesom_coarse_graining_dask_nc_T.zarr/").temp
W = xr.open_zarr("03_fesom_coarse_graining_dask_nc_W.zarr/").weights

In [31]:
groups = xr.open_dataset(
    "03_fesom_coarse_graining_dask_nc_groups.nc",
    chunks={"m": int(chunks_nodes // 10)},
).groups

groups

<xarray.DataArray 'groups' (m: 2200, j: 10)>
dask.array<open_dataset-groups, shape=(2200, 10), dtype=int64, chunksize=(200, 10), chunktype=numpy.ndarray>
Dimensions without coordinates: m, j

In [32]:
T_prime = (T * W).sel(n=groups).sum("j") / W.sel(n=groups).sum("j")
T_prime

<xarray.DataArray (m: 2200, t: 4000, z: 40)>
dask.array<truediv, shape=(2200, 4000, 40), dtype=float64, chunksize=(2200, 40, 10), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: m, t

In [33]:
T_prime = T_prime.compute()
T_prime

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 67.67 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray (m: 2200, t: 4000, z: 40)>
array([[[-3.08676286e-01,  3.16449283e-01,  1.46316817e-01, ...,
          1.55249225e-01,  9.40447577e-02,  2.30146461e-01],
        [-1.65925933e-01,  5.86507113e-02, -4.10068843e-01, ...,
          5.20058640e-01, -5.83393973e-01,  2.23317138e-01],
        [ 1.86031059e-01,  2.93808453e-01,  4.92714960e-03, ...,
         -1.38156715e-01, -4.18216078e-01, -3.15127103e-01],
        ...,
        [-3.87743410e-01,  7.03182326e-02,  9.27073519e-02, ...,
          2.23510256e-01, -8.48847661e-02,  2.31037937e-01],
        [ 7.01929496e-02, -3.50902307e-01, -1.48339021e-01, ...,
          3.83658000e-01, -3.18069928e-01, -3.84978534e-01],
        [ 6.95585517e-02,  6.93445097e-02, -2.50504278e-01, ...,
          8.62794986e-02, -2.39860102e-01, -3.50940761e-01]],

       [[ 5.59786117e-02,  1.17613393e-01, -3.81854829e-01, ...,
         -1.16171778e+00, -7.83778276e-02,  3.46329541e-01],
        [-2.48656558e-01,  2.00054187e-01,  1.03837780e-01, ...,
          5.83126951e-01, -3.58347538e-01,  9.78153545e-02],
        [-2.55024503e-01, -2.88191082e-01,  2.13427221e-04, ...,
         -1.37076830e-01, -5.45020690e-01, -6.91741861e-02],
...
        [ 3.17072365e-02,  3.08347384e-01,  1.13352698e-01, ...,
          5.51205321e-02,  3.15250410e-01,  3.50255602e-01],
        [-5.40361565e-01, -4.18526654e-01,  1.85165850e-01, ...,
          3.56884385e-02, -5.01861285e-01, -6.70486738e-02],
        [-3.19233777e-01,  3.11110176e-01, -7.79241870e-01, ...,
         -2.78850432e-01, -2.51747915e-01,  1.15544948e-01]],

       [[ 2.25394241e-02, -3.99767207e-01, -1.27931535e-01, ...,
          2.31856245e-01,  7.36722122e-02,  2.52854095e-01],
        [-1.47303606e-01,  4.76564710e-02, -2.68459936e-01, ...,
         -4.09542738e-01,  5.72681001e-02, -5.85299170e-01],
        [ 1.25959620e-01, -1.22321082e-01,  1.94032250e-01, ...,
         -5.67037767e-02,  2.99618059e-01,  1.50872848e-01],
        ...,
        [ 3.09221837e-01, -9.00113664e-02,  2.06428863e-01, ...,
          3.60548642e-01, -3.25548113e-01, -8.37620542e-02],
        [ 6.97350467e-02,  7.30192814e-02, -4.65607981e-01, ...,
         -1.12561905e-01,  1.61268041e-01, -1.17011439e-01],
        [ 2.58846463e-01,  3.51978112e-01,  2.11368891e-02, ...,
          1.80825261e-01,  3.36998250e-01, -5.41505554e-02]]])
Coordinates:
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: m, t